<a href="https://colab.research.google.com/github/aicrashcoursewinter24/ailab1BrianMann/blob/lab6/Copy_of_Copy_of_RAG_first_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install numexpr openai langchain_openai langchain langchain-community duckduckgo-search loguru qdrant-client sentence-transformers tiktoken docx2txt unstructured tqdm

In [ ]:
from typing import List, Dict, Any

import xml.etree.ElementTree as ET
import json

from langchain_core.documents import Document
from langchain_community.document_loaders.base import BaseLoader
from langchain_core.embeddings import Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Qdrant
from langchain.tools.retriever import create_retriever_tool


def parse_xml_to_dict(file_path: str) -> Dict[str, Any]:
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Function to parse each review element
    def parse_review(review_element: ET.Element) -> Dict[str, Any]:
        review_data = {}
        for child in review_element:
            if child.tag == 'reviewer':
                # Special handling for nested 'reviewer' tag
                reviewer_data = {grandchild.tag: grandchild.text for grandchild in child}
                review_data[child.tag] = reviewer_data
            else:
                review_data[child.tag] = child.text
        return review_data

    # Parse all reviews
    reviews = [parse_review(review) for review in root]
    return reviews


class XMLReviewLoader(BaseLoader):
    def __init__(self, file_path: str):
        self.file_path = file_path

    def load(self) -> List[Document]:
        docs = parse_xml_to_dict(self.file_path)
        return [Document(page_content=json.dumps(doc)) for doc in docs]


class VerboseEmbeddings(Embeddings):
    def __init__(self, embeddings: Embeddings):
        self.embeddings = embeddings

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        print(f"Embedding {len(texts)} documents")
        return self.embeddings.embed_documents(texts)

    def embed_query(self, text: str) -> List[float]:
        print(f"Embedding query: {text}")
        return self.embeddings.embed_query(text)


def build_vectorstore_from_xml_directory(path: str, vectorstore_path: str, loading_glob="**/*.xml",
                                         collection_name="my_documents",
                                         embeddings: Embeddings = VerboseEmbeddings(HuggingFaceEmbeddings())):
    # setting up a document loader which loads all xml reviews in the directory
    dir_loader = DirectoryLoader(path=path, glob=loading_glob, loader_cls=XMLReviewLoader,
                                 show_progress=True, use_multithreading=True)
    docs = dir_loader.load()
    # splitting the xml documents into chunks of 1000 characters with an overlap of 200 characters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000, chunk_overlap=100, add_start_index=True
    )
    all_splits: List[Document] = text_splitter.split_documents(docs)
    print(f"Split {len(docs)} documents into {len(all_splits)} chunks")
    # setting up a vectorstore with the Qdrant backend
    vectorstore = Qdrant.from_documents(
        all_splits,
        embeddings,
        path=vectorstore_path,
        collection_name=collection_name
    )
    # search args could return a different number of nearest neighbors.  Arbitrarily putting "6" here.
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
    return retriever


def retriever_tool(retriever):
    """Creates a tool that searches the vectorstore for documents similar to the input text."""
    tool = create_retriever_tool(
        retriever,
        "search_xml_documents",
        "Searches inside of our local store old Best Buy reviews.  " +
        "Useful for understanding how people talk about products from the early 2010's era",
    )
    return tool



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls drive/MyDrive/product_reviews

reviews_0001_24122_to_98772.xml      reviews_0024_5130266_to_5247642.xml
reviews_0002_98780_to_345240.xml     reviews_0025_5247643_to_5363104.xml
reviews_0003_345248_to_501823.xml    reviews_0026_5363109_to_5806838.xml
reviews_0004_501835_to_691133.xml    reviews_0027_5806868_to_6022633.xml
reviews_0005_691144_to_831905.xml    reviews_0028_6022641_to_6172272.xml
reviews_0006_831928_to_1117787.xml   reviews_0029_6172275_to_6358111.xml
reviews_0007_1117796_to_1382936.xml  reviews_0030_6358125_to_6396149.xml
reviews_0008_1382981_to_1597862.xml  reviews_0031_6396154_to_6423017.xml
reviews_0009_1597878_to_1906168.xml  reviews_0032_6423025_to_6486864.xml
reviews_0010_1906234_to_2063675.xml  reviews_0033_6486865_to_6587831.xml
reviews_0011_2063679_to_2151936.xml  reviews_0034_6587839_to_6701463.xml
reviews_0012_2151940_to_2284992.xml  reviews_0035_6701490_to_6801194.xml
reviews_0013_2285000_to_2440628.xml  reviews_0036_6801200_to_6889357.xml
reviews_0014_2440655_to_2799035.xml  reviews_0037_6

In [ ]:

review_dir = "drive/MyDrive/product_reviews"


In [ ]:
retriever = build_vectorstore_from_xml_directory(collection_name="test", path=review_dir, loading_glob="**/reviews_0001_24122_to_98772.xml", vectorstore_path='/content/reviews')


100%|██████████| 1/1 [00:01<00:00,  1.22s/it]


Split 5000 documents into 5024 chunks
Embedding 1 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documents
Embedding 64 documen

In [ ]:
review_tool = retriever_tool(retriever)

In [ ]:
import os
from getpass import getpass
from langchain.agents import AgentExecutor, AgentType, load_tools, initialize_agent
from langchain.chat_models import ChatAnyscale
from langchain_openai import OpenAIEmbeddings

In [ ]:
from google.colab import userdata

llm = ChatAnyscale(anyscale_api_base="https://api.endpoints.anyscale.com/v1",
                   anyscale_api_key=userdata.get("ANYSCALE_API_KEY"),
                   model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
                   temperature=0.7,
                   verbose=True)

In [ ]:
tools = load_tools(["llm-math", "ddg-search"], llm=llm) + [review_tool]

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
duckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
search_xml_documents: Searches inside of our local store old Best Buy reviews.  Useful for understanding how people talk about products from the early 2010's era

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, duckduckgo_search, search_xml_documents]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
agent.invoke("What were the best iphone accessories to buy in 2011 or so?")



> Entering new AgentExecutor chain...
 I need to look for the best iPhone accessories that were popular around 2011. I can use the `search_xml_documents` tool to search through old Best Buy reviews from that time period.

Action: search_xml_documents
Action Input: "iPhone accessories 2011"
ObservationEmbedding query: iPhone accessories 2011"
Observation

Observation: [Document(page_content='{"id": "69709", "sku": "8464084", "reviewer": {"name": "Polo"}, "aboutMe": null, "rating": "5.0", "title": "Very Well-Designed", "comment": "I had to get a velcro replacement for my Blackberry 7290 because the hard case that came with it was terrible. I am very active - chosing to run up three flights of stairs instead of taking the elevator. The phone kept falling out whenever I bent over or barely bumped it. When I got my new Blackberry 8830, I went ahead and got this new leather case with the very convenient magnetic flap. It\'s great! My new Blackberry never falls out! Getting my BB in and out

{'input': 'What were the best iphone accessories to buy in 2011 or so?',
 'output': "In 2011, some of the well-regarded iPhone accessories included the Apple MagSafe Charger, a protective case from OtterBox, and Apple's AirPods Pro (although the Pro model was released much later). Please note that this information is based on modern search results, and I couldn't find a definitive list specific to 2011."}

In [ ]:
agent.invoke("What is the hardest color for the human eye to see?")



> Entering new AgentExecutor chain...
 I don't have the ability to perceive things directly, so I need to rely on external sources to answer this question. I will use the duckduckgo_search tool to find more information.

Action: duckduckgo_search
Action Input: hardest color for the human eye to see
Observation
Observation: In this blog post, we'll delve into the fascinating topic of the hardest color for humans to see. From the rarity of colors to the limitations of our eyes, we'll explore why certain hues, particularly purple, are considered more difficult for the human eye to perceive. We'll also debunk common misconceptions about colors like black and ... We can reach into such impossible colors a little by fatiguing i.e. red and blue cones and then looking at a pure green light. On a final note, screens are horrible at making fully saturated colors. It's probably healthy to buy a physical color atlas and look though it from time to time. - Kevin Kostlan Nov 1, 2023 at 18:19 2 It 

{'input': 'What is the hardest color for the human eye to see?',
 'output': 'The hardest color for the human eye to see is generally considered to be purple due to its rarity in nature. However, if we focus on night vision, red becomes the most challenging color for the human eye to distinguish.'}

I added a new invoke statement. I think it is cool how it tries to come across as humman during its information retrieval process.